In [1]:
import os
import numpy as np
import pydicom

import pandas as pd
import sys
sys.path.append('/mnt/fast-disk1/mjc/utils_codes/read_weasis_raw_v0.96/')

import weasis_raw_data_api as wr
sys.path.append('/mnt/fast-disk1/mjc/utils_codes/')
from utils_test import *
from utils_metrics_3d import *

D_dir2header_df = {}
def get_dicom_header_df(image_dir , labels = []):
    global D_dir2header_df
    if image_dir in D_dir2header_df:
        return D_dir2header_df[image_dir]

    # image_dir = row['Image File Path']


    labels = ['ImageName','InstanceNumber',
            'BitsAllocated', 'BitsStored', 'Columns', 'HighBit', 
            'ImageOrientationPatient_0', 'ImageOrientationPatient_1', 'ImageOrientationPatient_2',
            'ImageOrientationPatient_3', 'ImageOrientationPatient_4', 'ImageOrientationPatient_5',
            'ImagePositionPatient_0', 'ImagePositionPatient_1', 'ImagePositionPatient_2',
            'Modality', 'PatientID', 'PhotometricInterpretation', 'PixelRepresentation',
            'PixelSpacing_0', 'PixelSpacing_1', 'RescaleIntercept', 'RescaleSlope', 'Rows', 'SOPInstanceUID',
            'SamplesPerPixel', 'SeriesInstanceUID', 'StudyID', 'StudyInstanceUID', 
            'WindowCenter', 'WindowWidth', 
        ] if not labels else labels

    data = {l: [] for l in labels}
    
    ctList = os.listdir(image_dir)
    ctList.sort()

    for image in ctList:
        if '.dcm' not in image:
            continue
        if os.path.getsize(os.path.join(image_dir, image)) < 5*1024:
            print('%s size < 5kb skiped!'%os.path.join(image_dir, image) )
            continue
        data["ImageName"].append(image)

        ds = pydicom.dcmread(os.path.join(image_dir, image))
        for metadata in ds.dir():
            if metadata not in data and metadata not in ['ImageOrientationPatient','ImagePositionPatient','PixelSpacing']:
                continue
            if metadata != "PixelData":
                metadata_values = getattr(ds, metadata)
                if type(metadata_values) == pydicom.multival.MultiValue and metadata not in ["WindowCenter", "WindowWidth"]:
                    for i, v in enumerate(metadata_values):
                        data[f"{metadata}_{i}"].append(v)  
                else:

                    if type(metadata_values) == pydicom.multival.MultiValue and metadata in ["WindowCenter", "WindowWidth"]:
                        data[metadata].append(metadata_values[0])
                    else:
                        
                        if metadata in ['ImageOrientationPatient','ImagePositionPatient','PixelSpacing']:
                            print( 'error of loading key: {}'.format(metadata) )                    
                        else:
                            data[metadata].append(metadata_values)

    df_image = pd.DataFrame(data).set_index("InstanceNumber")
    D_dir2header_df[image_dir] = df_image
    return df_image

In [2]:
def pd_str_replace(df , col, ori, new):
    if isinstance(col , str):
        try:
            df[col] = df[col].str.replace(ori,new, case = False) 
        except:
            pass
            
    elif isinstance(col, list):
        for one in col:
            pd_str_replace(df , one, ori, new)
    else:
        raise('col instance should be str or list')


def str_Xdrive2mnt(df_all):
    pd_str_replace(df_all, ['Image File Path' , 'Contour File Path'], "X:" , "/mnt/Y-drive")
    pd_str_replace(df_all, ['Image File Path' , 'Contour File Path'], r"\\" , "/")
    pd_str_replace(df_all, ['Image File Path'], "/mnt/Y-drive/ClinicalTrials/FNIH_VOLPACK", "/mnt/fast-disk1/mjc/AutoRecist/Inputs")
    pd_str_replace(df_all, ['Image File Path'], "/mnt/Y-drive/ClinicalTrialDone/FNIH_VOLPACK", "/mnt/fast-disk1/mjc/AutoRecist/Inputs")
    pd_str_replace(df_all, ['Image File Path'], "/mnt/Y-drive/ClinicalTrials", "/mnt/fast-disk1/mjc/AutoRecist/Inputs")

    pd_str_replace(df_all, ['Contour File Path'], "/mnt/Y-drive/ConvWeasisToRaw/PDS_AUTO_RECIST_Modified_By_Yen",
    "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw/PDS_AUTO_RECIST_Modified_By_Yen")
    pd_str_replace(df_all, ['Contour File Path'], "/mnt/Y-drive/ConvWeasisToRaw/PDS_AUTO_RECIST", "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw/PDS_AUTO_RECIST_RAW")
    pd_str_replace(df_all, ['Contour File Path'], "/mnt/Y-drive/ConvWeasisToRaw", "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw")
    pd_str_replace(df_all, ['Contour File Path'], "/mnt/Y-drive/ConvWeasisToMatlab", "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw")
    
def get_onect_from_list(df_list , ct):
    for i, df in enumerate(df_list):
        try:
            df_ct = df[ (df["Image File Path"]==ct) & (df['Location'].isin(['liver'])) ]
        except KeyError:
            df_ct = df[ (df["Image File Path"]==ct)]
        if df_ct.shape[0]:
            return df_ct , i
    print("warning! no CT was found")
    return None ,None

def raws2mask(raws , D_z_index, mask_vol = None):

    for raw in raws:

        radiologist_raw = wr.read(raw)
        slice_list = radiologist_raw.get_instance_number_array()
        if mask_vol is None:
            mask_vol = initialize_mask_vol(radiologist_raw , D_z_index)
        for j, one in enumerate(slice_list):
            mask = radiologist_raw.get_mask_image(j)
            mask_vol[D_z_index[one]] += mask
    return mask_vol

In [3]:
subsetname = 'Amgen'
folder = '/mnt/fast-data/mjc/AutoRECIST/Inputs/'

df_CTs = pd.read_excel(folder+'AutoRECIST_List_LesionSize_20220602_JM_SingleCTSeries.xlsx')
str_Xdrive2mnt(df_CTs)

df_CTs = df_CTs[df_CTs['dataset']==subsetname]
print(df_CTs)

                                Comments Patient ID  \
0                         minor revision      BAIJC   
1                         minor revision      BAIJD   
2    minor revision(lesion not in liver)      BAIJG   
3                         minor revision      BAIJL   
4                         major revision      BAIJM   
..                                   ...        ...   
148                       minor revision      BAITS   
149                       minor revision      BAITU   
150                       minor revision      BAITV   
151                       minor revision      BAITW   
152                       major revision      BAITX   

                                       Image File Path  \
0    /mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20...   
1    /mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20...   
2    /mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20...   
3    /mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20...   
4    /mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/

In [4]:
df_Yen = pd.read_excel(folder+'PDS_AUTO_RECIST CIA-LAB Testing Dataset Gold Standard_Yen_2022-06-13.xlsx')
str_Xdrive2mnt(df_Yen)

AI_raw_list = ['ScaleNAS9Slices_ToRaw_Test353.csv',]
df_AIs = []
for one in AI_raw_list:
    df = pd.read_csv(one)
    str_Xdrive2mnt(df)
    df_AIs.append(df)




Metrics_vol = []
metrics_save_path = 'Metrics_%s_vs_Yen_%s.csv'%('ScaleNAS9Slices' ,subsetname )
CTs = df_CTs["Image File Path"].values.tolist()

for ct in CTs:
    df_image = get_dicom_header_df( ct )
    instanceNumber_list = df_image.index.to_list()
    D_z_index = instanceNumber2Matrix_z_index(instanceNumber_list)


    df_ct_Yen = df_Yen[df_Yen["Image File Path"]==ct]
    df_ct_AI , dataset_id = get_onect_from_list(df_AIs , ct)
    # break
    if (df_ct_AI is None):
        if df_ct_Yen.shape[0]:
            fn = df_ct_Yen.shape[0]
            print('{} has {} FNs!'.format(ct , fn))   
        continue

    if not df_ct_Yen.shape[0]:
        fp = df_ct_AI.shape[0]
        print( '{} has {} FPs'.format(ct , fp)  )
        continue
    else:
        print(ct)

    raws = df_ct_AI["Contour File Path"].values.tolist()
    vol_pred = raws2mask(raws , D_z_index, mask_vol = None)
    connectivity = 2
    from skimage import measure
    labels_pred=measure.label(vol_pred,connectivity=connectivity)
    l_pred,c_pred = np.unique(labels_pred , return_counts=True)
    ix2 = l_pred>0
    l_pred = l_pred[ix2] #background pixels are labeled as 0, so we exclude them
    c_pred = c_pred[ix2]
    if len(l_pred)!= len(raws):
        print("warning! raws overlaped on {}".format(ct))


    for _ , row in df_ct_Yen.iterrows():

        Yen_raw = wr.read(row['Contour File Path'])
        gt_vol = initialize_mask_vol(Yen_raw , D_z_index)

        slice_list = Yen_raw.get_instance_number_array()
        for j, one in enumerate(slice_list):
            mask = Yen_raw.get_mask_image(j)
            gt_vol[D_z_index[one]] = mask
        
        hit = vols_seg_results(gt_vol , vol_pred, CTname=row['Contour File Path'], gt_keep_largest=1)
        Metrics_vol.extend(hit)

        _n = len(Metrics_vol)
        if _n%100==0 or _n in [1,2,5,10,30,50]:
            df_metrics = pd.DataFrame(Metrics_vol, 
                                    columns = ['file_name','igt','merge','#gt','#pred',
                                                'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                                'area_gt','area_pred','intersection','union']) 
            df_metrics.to_csv(metrics_save_path)
        

/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20020408/BAIJC/D2004_02_27/E20040227/CT/S0002
/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20020408/BAIJD/D2004_02_07/E20040207/CT/S0002
/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20020408/BAIJG/D2004_02_02/E20040202/CT/S0013
/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20020408/BAIJL/D2004_02_24/E20040224/CT/S3464
/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20020408/BAIJM/D2004_01_31/E20040131/CT/S0003
/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20020408/BAIJN/D2004_02_03/E20040203/CT/S0004
/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20020408/BAIJO/D2004_02_19/E20040219/CT/S0005
/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20020408/BAIJP/D2004_02_26/E20040226/CT/S0002
/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20020408/BAIJQ/D2004_04_27/E20040427/CT/S0007
/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20020408/BAIJR/D2004_02_11/E20040211/CT/S0006
/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20020408/BAIJT/D2004_02_27/E20040227/CT/S0002
/mnt/fast-

In [5]:
df_metrics = pd.DataFrame(Metrics_vol, 
                        columns = ['file_name','igt','merge','#gt','#pred',
                                    'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                    'area_gt','area_pred','intersection','union']) 
df_metrics.to_csv(metrics_save_path)

In [6]:
pd.options.display.float_format = "{:.3f}".format
df_metrics = pd.DataFrame(Metrics_vol, 
                          columns = ['file_name','igt','merge','#gt','#pred',
                                     'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                     'area_gt','area_pred','intersection','union']) 
df_metrics.describe([.05, .25, .5, .75, .95])

,igt,merge,#gt,#pred,iou_score,dice_score,over_seg,under_seg,area_gt,area_pred,intersection,union
count,1091.000,1091.000,1091.000,1091.000,1091.000,1091.000,1091.000,1091.000,1091.000,1091.000,1091.000,1091.000
mean,1.014,0.000,1.000,10.329,0.387,0.489,43.860,0.385,23399.610,42796.142,19156.788,47038.964
std,0.117,0.000,0.000,5.959,0.292,0.335,615.303,0.379,87756.513,142555.328,74384.571,151092.511
min,1.000,0.000,1.000,1.000,0.000,0.000,0.000,0.000,14.000,0.000,0.000,18.000
5%,1.000,0.000,1.000,3.000,0.000,0.000,0.000,0.003,63.500,0.000,0.000,75.500
25%,1.000,0.000,1.000,5.000,0.033,0.063,0.017,0.057,296.000,174.500,81.500,503.000
50%,1.000,0.000,1.000,8.000,0.425,0.596,0.217,0.217,1298.000,1479.000,759.000,2344.000
75%,1.000,0.000,1.000,16.000,0.643,0.783,0.763,0.743,6407.500,10778.000,4534.000,12775.000
95%,1.000,0.000,1.000,22.000,0.814,0.898,13.610,1.000,112753.500,248480.000,96008.500,276860.000
max,2.000,0.000,1.000,23.000,0.903,0.949,18033.245,1.000,926709.000,1113120.000,844938.000,1179612.000


In [7]:
df_metrics[df_metrics.dice_score>0.25].describe([.05, .25, .5, .75, .95])

,igt,merge,#gt,#pred,iou_score,dice_score,over_seg,under_seg,area_gt,area_pred,intersection,union
count,766.000,766.000,766.000,766.000,766.000,766.000,766.000,766.000,766.000,766.000,766.000,766.000
mean,1.012,0.000,1.000,10.339,0.545,0.684,0.635,0.209,31794.782,38885.753,26897.277,43783.258
std,0.108,0.000,0.000,6.047,0.194,0.174,0.904,0.216,102053.823,125508.376,87606.268,138135.164
min,1.000,0.000,1.000,1.000,0.143,0.251,0.000,0.000,24.000,26.000,10.000,51.000
5%,1.000,0.000,1.000,3.000,0.212,0.349,0.018,0.003,145.750,169.000,92.750,225.250
25%,1.000,0.000,1.000,5.000,0.398,0.569,0.124,0.042,613.750,704.750,439.250,932.750
50%,1.000,0.000,1.000,8.000,0.555,0.714,0.307,0.117,2257.000,2619.500,1648.000,3332.000
75%,1.000,0.000,1.000,16.000,0.707,0.828,0.757,0.318,9991.750,12202.000,8257.500,15342.750
95%,1.000,0.000,1.000,22.000,0.833,0.909,2.552,0.676,178556.500,229222.500,155508.000,242935.250
max,2.000,0.000,1.000,23.000,0.903,0.949,5.637,0.846,926709.000,1113120.000,844938.000,1179612.000


In [8]:
dfinnermerge = pd.merge(df_metrics,df_Yen,how='inner',left_on='file_name' , right_on='Contour File Path')
for col in dfinnermerge.columns.tolist():
    print(col , len(set(dfinnermerge[col].tolist() )) )



pts = dfinnermerge["Image File Path"].values.tolist()
FPs = []
for onept in list(set(pts)):
    df_onept = dfinnermerge[dfinnermerge["Image File Path"]==onept]
    assert( min(df_onept["#pred"]) == max(df_onept["#pred"]))
    fp = max(df_onept["#pred"])
    FPs.append(fp)
print("="*80)
print( "In total, {} CT series; {} AI detections ".format( len(FPs) , sum(FPs) ) )

dices = dfinnermerge.dice_score.tolist()

for th in [0, 0.1, 0.2, 0.25, 0.5]:
    TP = [p>th for p in dices ]
    assert( len(TP) == len(dices))
    fprate = ( sum(FPs) - sum(TP) ) / len(FPs)
    print(f"sensitivity is {sum(TP)/len(dices):.3f}({sum(TP)}/{len(dices)}) FP-rate is {fprate:.1f} per CT-serie at threshold {th}")

file_name 1091
igt 2
merge 1
#gt 1
#pred 20
iou_score 870
dice_score 870
over_seg 854
under_seg 826
area_gt 937
area_pred 738
intersection 795
union 987
Image File Path 153
Contour File Path 1091
Raw File Name 1091
Uni 1054
Perp 1042
Bi 1088
Volume 1085
In total, 153 CT series; 1070 AI detections 
sensitivity is 0.797(869/1091) FP-rate is 1.3 per CT-serie at threshold 0
sensitivity is 0.740(807/1091) FP-rate is 1.7 per CT-serie at threshold 0.1
sensitivity is 0.719(784/1091) FP-rate is 1.9 per CT-serie at threshold 0.2
sensitivity is 0.702(766/1091) FP-rate is 2.0 per CT-serie at threshold 0.25
sensitivity is 0.582(635/1091) FP-rate is 2.8 per CT-serie at threshold 0.5


In [10]:
#subgroup analysis for gt_lesion >=10mm

Uni_thresh = 10
df_subgroup = dfinnermerge[dfinnermerge['Uni']>=Uni_thresh]
df_subgroup.describe([.05, .25, .5, .75, .95])


,igt,merge,#gt,#pred,iou_score,dice_score,over_seg,under_seg,area_gt,area_pred,intersection,union,Uni,Perp,Bi,Volume
count,888.000,888.000,888.000,888.000,888.000,888.000,888.000,888.000,888.000,888.000,888.000,888.000,888.000,888.000,888.000,888.000
mean,1.011,0.000,1.000,9.963,0.430,0.541,21.366,0.333,28720.679,49947.474,23522.291,55145.863,38.889,26.014,1729.711,63659.279
std,0.106,0.000,0.000,5.850,0.281,0.315,232.207,0.351,96495.449,150418.399,81834.155,160087.640,35.853,21.462,3680.136,200603.086
min,1.000,0.000,1.000,1.000,0.000,0.000,0.000,0.000,58.000,0.000,0.000,58.000,10.012,3.232,40.788,151.447
5%,1.000,0.000,1.000,3.000,0.000,0.000,0.000,0.003,226.050,0.000,0.000,288.350,11.180,8.194,91.102,540.570
25%,1.000,0.000,1.000,5.000,0.176,0.300,0.058,0.047,734.000,572.750,322.500,1143.750,16.496,12.180,204.231,1650.955
50%,1.000,0.000,1.000,8.000,0.474,0.643,0.240,0.167,2303.000,2697.500,1256.000,3828.500,27.108,19.021,527.305,5828.670
75%,1.000,0.000,1.000,14.000,0.677,0.807,0.781,0.564,9441.750,16024.750,6983.000,17393.500,45.651,31.496,1356.984,22046.600
95%,1.000,0.000,1.000,22.000,0.822,0.902,10.448,1.000,158748.500,276634.000,124034.100,335721.600,110.333,73.058,8431.441,324323.850
max,2.000,0.000,1.000,23.000,0.903,0.949,4661.493,1.000,926709.000,1113120.000,844938.000,1179612.000,254.088,146.833,36132.810,1979810.000


In [11]:
df_subgroup[df_subgroup.dice_score>0.25].describe([.05, .25, .5, .75, .95])

,igt,merge,#gt,#pred,iou_score,dice_score,over_seg,under_seg,area_gt,area_pred,intersection,union,Uni,Perp,Bi,Volume
count,683.000,683.000,683.000,683.000,683.000,683.000,683.000,683.000,683.000,683.000,683.000,683.000,683.000,683.000,683.000,683.000
mean,1.010,0.000,1.000,10.092,0.553,0.691,0.602,0.206,35637.968,43580.211,30149.962,49068.217,42.836,28.534,2051.081,78171.087
std,0.101,0.000,0.000,5.943,0.193,0.172,0.859,0.217,107452.235,132158.004,92255.676,145414.524,38.636,23.049,4049.452,218653.574
min,1.000,0.000,1.000,1.000,0.145,0.254,0.000,0.000,76.000,32.000,27.000,114.000,10.012,4.259,43.224,261.527
5%,1.000,0.000,1.000,3.000,0.221,0.362,0.018,0.003,307.500,284.000,205.500,483.200,11.591,8.585,100.387,662.586
25%,1.000,0.000,1.000,5.000,0.408,0.580,0.117,0.040,940.500,1121.500,694.500,1412.500,17.875,13.200,237.971,2165.385
50%,1.000,0.000,1.000,8.000,0.564,0.722,0.287,0.114,3240.000,3406.000,2213.000,4268.000,29.754,21.015,614.766,7734.690
75%,1.000,0.000,1.000,16.000,0.718,0.836,0.709,0.309,12746.000,16044.500,9790.500,17803.500,50.418,34.235,1675.547,36003.200
95%,1.000,0.000,1.000,22.000,0.836,0.911,2.472,0.679,203190.700,250464.800,180266.600,276425.200,132.791,84.087,9624.749,437800.500
max,2.000,0.000,1.000,23.000,0.903,0.949,5.616,0.846,926709.000,1113120.000,844938.000,1179612.000,254.088,146.833,36132.810,1979810.000


In [12]:
def detection_performance(dfinnermerge):
    pts = dfinnermerge["Image File Path"].values.tolist()
    FPs = []
    for onept in list(set(pts)):
        df_onept = dfinnermerge[dfinnermerge["Image File Path"]==onept]
        assert( min(df_onept["#pred"]) == max(df_onept["#pred"]))
        fp = max(df_onept["#pred"])
        FPs.append(fp)
    print("="*80)
    print( "In total, {} CT series; {} AI detections ".format( len(FPs) , sum(FPs) ) )

    dices = dfinnermerge.dice_score.tolist()

    for th in [0, 0.1, 0.2, 0.25, 0.5]:
        TP = [p>th for p in dices ]
        assert( len(TP) == len(dices))
        fprate = ( sum(FPs) - sum(TP) ) / len(FPs)
        print(f"sensitivity is {sum(TP)/len(dices):.3f}({sum(TP)}/{len(dices)}) FP-rate is {fprate:.1f} per CT-serie at threshold {th}")

In [14]:
detection_performance(df_subgroup)

In total, 152 CT series; 1065 AI detections 
sensitivity is 0.864(767/888) FP-rate is 2.0 per CT-serie at threshold 0
sensitivity is 0.810(719/888) FP-rate is 2.3 per CT-serie at threshold 0.1
sensitivity is 0.786(698/888) FP-rate is 2.4 per CT-serie at threshold 0.2
sensitivity is 0.769(683/888) FP-rate is 2.5 per CT-serie at threshold 0.25
sensitivity is 0.649(576/888) FP-rate is 3.2 per CT-serie at threshold 0.5
